In [ ]:
from typing import Annotated,Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage


In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()


In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph-Modules")

In [ ]:
class SubState(TypedDict):
    name : str
    family_name : str

def subgraph_node_1(state:SubState):
    return SubState({"family_name":"LEE"})

def subgraph_node_2(state:SubState):
    return SubState({"name":f"{state['name']} {state['family_name']}"})


state_graph = StateGraph(SubState)
state_graph.add_node('sub1',subgraph_node_1)
state_graph.add_node('sub2',subgraph_node_2)
state_graph.add_edge(START, "sub1")
state_graph.add_edge("sub1", "sub2")
state_graph.add_edge("sub2",END)
subgraph = state_graph.compile()
from langchain_teddynote.graphs import visualize_graph

visualize_graph(subgraph, xray=True)

In [ ]:
class priState(TypedDict):
    name : str

def primary_node_1(state:priState):
    return priState({"name":f"My name is {state['name']}"})





state_graph = StateGraph(priState)
state_graph.add_node('pri1',primary_node_1)
state_graph.add_node('pri2',subgraph)
state_graph.add_edge(START, "pri1")
state_graph.add_edge("pri1", "pri2")
state_graph.add_edge("pri2",END)
prigraph = state_graph.compile()
from langchain_teddynote.graphs import visualize_graph

visualize_graph(prigraph, xray=True)

In [18]:
# 그래프 스트리밍 처리를 통한 서브그래프 데이터 청크 단위 순차 출력
# subgraphs 파라미터를 True로 설정하여 하위 그래프 포함 스트리밍 처리
for chunk in prigraph.stream({"name": "Teddy"}, subgraphs=True):
    print(chunk)

((), {'pri1': {'name': 'My name is Teddy'}})
(('pri2:d91c6802-79b1-824c-0917-1ff939548746',), {'sub1': {'family_name': 'LEE'}})
(('pri2:d91c6802-79b1-824c-0917-1ff939548746',), {'sub2': {'name': 'My name is Teddy LEE'}})
((), {'pri2': {'name': 'My name is Teddy LEE'}})
